In [47]:
# ! pip install openai
# ! pip install multion

In [1]:
import os
from openai import OpenAI
from multion.client import MultiOn

In [2]:
os.environ["OPENAI_API_KEY"] = "api-key"

In [3]:
client = OpenAI()
client_multion = MultiOn(api_key="apikey")

In [4]:
temperature = 0.0

In [15]:
# user_prompt = """
# Shopping for a dress.
# """

user_prompt = """
Play Lose Yourself by Eminem on Spotify.
"""

In [16]:
system_prompt_llm1_init = """
You are an assistant that will take a web navigation task as an input and output
a detailed chain of thought set of instructions to complete this task. If you
require more information than what is given in the original prompt, your output
should include asking this question as one of the steps. The question should be
formatted using "quotation marks" so that it is easier to follow. After you give a response, you will be given
feedback by another teacher assistant to iteratively improve your response.
Essentially, you are the actor in an actor-critic model. The goal is to
create a set of instructions for another LLM agent to perform using web navigation,
so the response should be framed for an LLM, not a human. For context, the LLM has
access to Google Chrome and maybe other applications on the user's laptop.

Types of questions to ask the user are about:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

This clarification may be needed because the user of this product is not an expert
at prompting or specificity. For example, if the user wants to watch sports clips,
you can ask which sport. Then, if they say soccer, ask if they want recent highlights or
soccer clips from any specific time period. Clarifying user intent is crucial.

RESPONSE SHOULD BE IN LIST FORMAT, WHERE EACH ELEMENT HAS A SHORT RELEVANT TITLE AND CORRESPONDING DESCRIPTION:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: ORIGINAL PROMPT FROM USER ,
<<<TITLE-DESCRIPTION>>>: DESCRIPTION OF STEP ,
<<<ASK-QUESTION>>>: "QUESTION TO ASK TO THE USER OF THIS PRODUCT?" ,
``MORE LINES HERE``
]
"""

system_prompt_llm2_init = """
You are an assistant that will take the proposed chain of thought for completing
a web navigation task and output constructive criticism feedback for the chain
of thought. Your job is to find flaws in the chain of thought logic, misordered
steps, or unnecessary steps. Essentially, you are the critic in an actor-critic model.

Some of the questions that the actor asks the user are crucial, like the following:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

That being said, steps should not be redundant or unnecessary.

INPUT FORMAT:
["{'role': 'user', 'content': 'Initial user prompt here.'}",
 '{\'role\': \'assistant\', \'content\': \'[
  \\n<<<TITLE-OF-STEP>>>: Step description ,
  \\n<<<ASK-QUESTION>>>: "Question to user here (should be as specific as possible)?"
  `MORE-LINES-HERE`'}',']

OUTPUT FORMAT:
Here is some feedback for the chain of thought so far:
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
`MORE-LINES-HERE`
"""
messages = [{"role": "user", "content": user_prompt}]

In [17]:
# LLM 1 (Initial Proposal)
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role": "system", "content": system_prompt_llm1_init}] + messages,
  temperature=temperature,
)

response = completion.choices[0].message.content
messages.append({"role": "assistant", "content": response})
print("PROPOSED CHAIN OF THOUGHT:")
print(response)
print("\n\n")

for i in range(3):
  # LLM 2
  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_prompt_llm2_init}, {"role": "user", "content": "\n".join(list(map(str, [messages[0], messages[-1]])))}],
    temperature=temperature,
  )

  response = completion.choices[0].message.content
  messages.append({"role": "user", "content": response})
  print("FEEDBACK")
  print(response)
  print("\n\n")

  # LLM 1
  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_prompt_llm1_init}] + messages,
    temperature=temperature,
  )

  response = completion.choices[0].message.content
  messages.append({"role": "assistant", "content": response})
  print("PROPOSED CHAIN OF THOUGHT:")
  print(response)
  print("\n\n")

final_prompt = messages[-1]["content"]

PROPOSED CHAIN OF THOUGHT:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: Check NVIDIA stock price on July 13th 2024 ,
<<<DESCRIPTION-OF-STEP-AS-TITLE>>>: Open a Web Browser ,
<<<DESCRIPTION>>>: Launch Google Chrome or any other web browser available on the user's laptop. ,
<<<DESCRIPTION-OF-STEP-AS-TITLE>>>: Navigate to a Financial Website ,
<<<DESCRIPTION>>>: Go to a financial news or stock market website such as Yahoo Finance, Google Finance, or Bloomberg. ,
<<<DESCRIPTION-OF-STEP-AS-TITLE>>>: Search for NVIDIA Stock ,
<<<DESCRIPTION>>>: Use the search bar on the financial website to enter "NVIDIA" or "NVDA" to find the stock information. ,
<<<DESCRIPTION-OF-STEP-AS-TITLE>>>: Access Historical Data ,
<<<DESCRIPTION>>>: Look for an option that allows you to view historical stock prices, often found under a "Historical Data" or "Price History" tab. ,
<<<DESCRIPTION-OF-STEP-AS-TITLE>>>: Set the Date Range ,
<<<DESCRIPTION>>>: Input the date range to include July 13th, 2024, ensuring that the start

In [8]:
create_response = client_multion.sessions.create(
    url="https://google.com/",
    mode="standard",
    use_proxy=True
)

In [9]:
step_responses = []
max_iters = 20

current_prompt = final_prompt
step_response = client_multion.sessions.step(
    session_id=create_response.session_id,
    cmd=current_prompt,
    include_screenshot=True
)
step_responses.append(step_response)
status = step_response.status
print(step_response)

for i in range(max_iters-1):
    if status not in ["CONTINUE", "DONE"]:
        temp_input = input()
        current_prompt = current_prompt if len(temp_input) == 0 else temp_input
        step_response = client_multion.sessions.step(
            session_id=create_response.session_id,
            cmd=current_prompt,
            include_screenshot=True
        )
    elif status == "DONE":
        break
    elif status == "CONTINUE":
        step_response = client_multion.sessions.step(
            session_id=create_response.session_id,
            cmd=current_prompt,
            include_screenshot=True
        )
    step_responses.append(step_response)
    status = step_response.status
    print(step_response)

status='NOT_SURE' message='What occasion or event are you shopping for a dress for?\n' session_id='162c3540-d7c3-452c-8248-d122ac620aa0' url='https://www.google.com/' screenshot='https://multion-client-screenshots.s3.us-east-2.amazonaws.com/162c3540-d7c3-452c-8248-d122ac620aa0_1721529411705_screenshot.png' metadata=SessionStepSuccessMetadata(temperature=0.2)
college party
status='NOT_SURE' message='What is your budget for the dress?\n' session_id='162c3540-d7c3-452c-8248-d122ac620aa0' url='https://www.google.com/' screenshot='https://multion-client-screenshots.s3.us-east-2.amazonaws.com/162c3540-d7c3-452c-8248-d122ac620aa0_1721529422530_screenshot.png' metadata=SessionStepSuccessMetadata(temperature=0.2)
under 40 bucks
status='NOT_SURE' message='What are your style preferences, including color, length, pattern, and size?\n' session_id='162c3540-d7c3-452c-8248-d122ac620aa0' url='https://www.google.com/' screenshot='https://multion-client-screenshots.s3.us-east-2.amazonaws.com/162c3540-d

In [10]:
system_prompt_llm1 = """
You are an assistant that will take a web navigation task as an input and output
a detailed chain of thought set of instructions to complete this task.

Complete set of inputs:
1) Initial user prompt
2) Chain of thought produced to complete initial prompt
3) Log messages from an LLM agent trying to complete the web navigation task
based on the given prompt and chain of thought.

Your task is to determine whether the task has been completed or not. If the
task has not been completed, create a chain of thought to complete the task
from the current state. If the task is completed, output: "DONE-WITH-TASK"

If the task is not completed:
If you require more information than what is given in the original prompt, your output
should include asking this question as one of the steps. The question should be
formatted using "quotation marks" so that it is easier to follow. After you give a response, you will be given
feedback by another teacher assistant to iteratively improve your response.
Essentially, you are the actor in an actor-critic model. The goal is to
create a set of instructions for another LLM agent to perform using web navigation,
so the response should be framed for an LLM, not a human. For context, the LLM has
access to Google Chrome and maybe other applications on the user's laptop.

Types of questions to ask the user are about:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

This clarification may be needed because the user of this product is not an expert
at prompting or specificity. For example, if the user wants to watch sports clips,
you can ask which sport. Then, if they say soccer, ask if they want recent highlights or
soccer clips from any specific time period. Clarifying user intent is crucial.

RESPONSE SHOULD BE IN LIST FORMAT, WHERE EACH ELEMENT HAS A SHORT RELEVANT TITLE AND CORRESPONDING DESCRIPTION:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: ORIGINAL PROMPT FROM USER ,
<<<TITLE-DESCRIPTION>>>: DESCRIPTION OF STEP ,
<<<ASK-QUESTION>>>: "QUESTION TO ASK TO THE USER OF THIS PRODUCT?" ,
``MORE LINES HERE``
]
"""

system_prompt_llm2 = """
You are an assistant that will take the proposed chain of thought for completing
a web navigation task and output constructive criticism feedback for the chain
of thought.

Complete set of inputs:
1) Initial user prompt
2) Chain of thought produced to complete initial prompt
3) Log messages from an LLM agent trying to complete the web navigation task
based on the given prompt and chain of thought.
4) A proposed chain of thought to complete the task in the current state.

Your job is to find flaws in the proposed chain of thought logic, misordered
steps, or unnecessary steps. Essentially, you are the critic in an actor-critic model.

Some of the questions that the actor asks the user are crucial, like the following:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

That being said, steps should not be redundant or unnecessary.

INPUT FORMAT FOR PROPOSED CHAIN OF THOUGHT:
["{'role': 'user', 'content': 'Initial user prompt here.'}",
 '{\'role\': \'assistant\', \'content\': \'[
  \\n<<<TITLE-OF-STEP>>>: Step description ,
  \\n<<<ASK-QUESTION>>>: "Question to user here (should be as specific as possible)?"
  `MORE-LINES-HERE`'}',']

OUTPUT FORMAT:
Here is some feedback for the chain of thought so far:
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
`MORE-LINES-HERE`
"""
llm1_prompt = "Original chain of thought:\n" + final_prompt
llm1_prompt += "\nLog messages from LLM attempt at solving task using chain of thought:\n" + "\n".join([i.__repr__() for i in step_responses])
# llm2_prompt = llm1_prompt + "\nProposed chain of thought to solve task from current state:\n"
messages = [{"role": "user", "content": llm1_prompt}]

In [11]:
while messages[-1]["role"] != "assistant" or "DONE-WITH-TASK" not in messages[-1]["content"]:
    # LLM 1
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role": "system", "content": system_prompt_llm1}] + messages,
      temperature=temperature,
    )

    response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": response})
    print("PROPOSED CHAIN OF THOUGHT:")
    print(response)
    print("\n\n")

    for i in range(3):
      # LLM 2
      completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt_llm2}, {"role": "user", "content": "\n".join(list(map(str, [messages[0], "Proposed chain of thought to solve task from current state:", messages[-1]])))}],
        temperature=temperature,
      )

      response = completion.choices[0].message.content
      messages.append({"role": "user", "content": response})
      print("FEEDBACK")
      print(response)
      print("\n\n")

      # LLM 1
      completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt_llm1}] + messages,
        temperature=temperature,
      )

      response = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": response})
      print("PROPOSED CHAIN OF THOUGHT:")
      print(response)
      print("\n\n")

    final_prompt = messages[-1]["content"]

    current_prompt = final_prompt
    step_response = client_multion.sessions.step(
        session_id=create_response.session_id,
        cmd=current_prompt,
        include_screenshot=True
    )
    step_responses.append(step_response)
    status = step_response.status
    print(step_response)

    for i in range(max_iters-1):
        if status not in ["CONTINUE", "DONE"]:
            temp_input = input()
            current_prompt = current_prompt if len(temp_input) == 0 else temp_input
            step_response = client_multion.sessions.step(
                session_id=create_response.session_id,
                cmd=current_prompt,
                include_screenshot=True
            )
        elif status == "DONE":
            break
        elif status == "CONTINUE":
            step_response = client_multion.sessions.step(
                session_id=create_response.session_id,
                cmd=current_prompt,
                include_screenshot=True
            )
        step_responses.append(step_response)
        status = step_response.status
        print(step_response)

PROPOSED CHAIN OF THOUGHT:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: Shopping for a dress ,
<<<CONFIRM-VERIFICATION>>>: The LLM has reached a point where eBay is asking for verification to proceed with the purchase. The user needs to complete this verification step on their end. ,
<<<ASK-QUESTION>>>: "Please complete the verification process on eBay and let me know once it's done so I can continue assisting you with the purchase." ,
<<<WAIT-FOR-USER-RESPONSE>>>: Wait for the user to confirm that they have completed the verification process before proceeding with the next steps. ,
<<<PROCEED-TO-PURCHASE>>>: Once the user confirms the verification is complete, proceed to add the selected dress to the cart and guide the user through the checkout process. ,
<<<REQUEST-CREDENTIALS-IF-NEEDED>>>: If the user has not provided their eBay credentials yet, ask for their username and password to complete the purchase. ,
<<<ASK-QUESTION>>>: "Please provide your username and password for eBay if you intend

KeyboardInterrupt: 